In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')


In [3]:
df

,ID,STAY_DRG_CD,STAY_FROM_DT,STAY_THRU_DT,STUS_CD,TYPE_ADM,SRC_ADMS,AD_DGNS,DGNSCD01,PRCDRCD01,...,DGNSCD22,PRCDRCD22,DGNSCD23,PRCDRCD23,DGNSCD24,PRCDRCD24,DGNSCD25,PRCDRCD25,stay_drg_cd,Readmitted_30
0,17319,NaN,12/13/2017 0:00,12/20/2017 0:00,62,1,2,M25551,S72001A,0SRR01Z,...,Z803,NaN,Z86711,NaN,Z86718,NaN,Z85828,NaN,469,0
1,19722,NaN,10/19/2017 0:00,10/23/2017 0:00,1,1,1,R531,A419,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,871,1
2,89699,NaN,8/6/2018 0:00,8/8/2018 0:00,1,1,1,R002,J690,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177,0
3,8086,NaN,12/20/2016 0:00,12/27/2016 0:00,62,5,1,K661,K661,NaN,...,I440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,393,0
4,68049,NaN,1/6/2016 0:00,1/12/2016 0:00,6,1,1,J9601,J690,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130291,119880,NaN,12/18/2018 0:00,12/19/2018 0:00,1,3,2,R079,T82855A,027035Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,0
130292,103695,NaN,11/23/2019 0:00,11/27/2019 0:00,6,1,1,R531,I130,3E02340,...,Z79899,NaN,Z23,NaN,I2781,NaN,NaN,NaN,291,1
130293,131933,NaN,12/10/2020 0:00,12/17/2020 0:00,6,1,1,R197,A419,0DBN8ZX,...,Z9049,NaN,Z90710,NaN,NaN,NaN,NaN,NaN,871,0
130294,146868,NaN,10/10/2022 0:00,10/19/2022 0:00,3,1,1,R4182,A4159,0BH17EZ,...,R7401,NaN,F209,NaN,I071,NaN,Z9981,NaN,871,0


In [4]:
def count_diagnoses(df):
    # Find all diagnosis columns
    dgns_cols = [col for col in df.columns if 'DGNSCD' in col]
    
    # Count non-empty diagnoses
    df['DGNS_COUNT'] = df[dgns_cols].notna().sum(axis=1)
    
    return df

In [5]:
def diagnoses_frequency(df):
    # Find all diagnosis columns
    dgns_cols = [col for col in df.columns if 'DGNSCD' in col]
    
    # Stack all diagnosis columns into a single series
    all_diagnoses = df[dgns_cols].stack().reset_index()
    all_diagnoses.columns = ['row_index', 'column', 'diagnosis_code']
    
    # Drop missing values
    all_diagnoses = all_diagnoses.dropna()
    
    # Count frequency of each diagnosis code
    diagnosis_counts = all_diagnoses['diagnosis_code'].value_counts().reset_index()
    diagnosis_counts.columns = ['diagnosis_code', 'frequency']
    
    return diagnosis_counts

In [6]:
df = count_diagnoses(df)

In [8]:
def count_procedures(df):
    # Find all procedure columns
    prcdr_cols = [col for col in df.columns if 'PRCDRCD' in col]
    
    # Count non-empty procedures
    df['PRCDR_COUNT'] = df[prcdr_cols].notna().sum(axis=1)
    
    return df

In [9]:
df = count_procedures(df)

In [10]:
df

,ID,STAY_DRG_CD,STAY_FROM_DT,STAY_THRU_DT,STUS_CD,TYPE_ADM,SRC_ADMS,AD_DGNS,DGNSCD01,PRCDRCD01,...,DGNSCD23,PRCDRCD23,DGNSCD24,PRCDRCD24,DGNSCD25,PRCDRCD25,stay_drg_cd,Readmitted_30,DGNS_COUNT,PRCDR_COUNT
0,17319,NaN,12/13/2017 0:00,12/20/2017 0:00,62,1,2,M25551,S72001A,0SRR01Z,...,Z86711,NaN,Z86718,NaN,Z85828,NaN,469,0,25,1
1,19722,NaN,10/19/2017 0:00,10/23/2017 0:00,1,1,1,R531,A419,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,871,1,15,0
2,89699,NaN,8/6/2018 0:00,8/8/2018 0:00,1,1,1,R002,J690,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,177,0,21,0
3,8086,NaN,12/20/2016 0:00,12/27/2016 0:00,62,5,1,K661,K661,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,393,0,22,0
4,68049,NaN,1/6/2016 0:00,1/12/2016 0:00,6,1,1,J9601,J690,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,177,0,13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130291,119880,NaN,12/18/2018 0:00,12/19/2018 0:00,1,3,2,R079,T82855A,027035Z,...,NaN,NaN,NaN,NaN,NaN,NaN,247,0,7,3
130292,103695,NaN,11/23/2019 0:00,11/27/2019 0:00,6,1,1,R531,I130,3E02340,...,Z23,NaN,I2781,NaN,NaN,NaN,291,1,24,7
130293,131933,NaN,12/10/2020 0:00,12/17/2020 0:00,6,1,1,R197,A419,0DBN8ZX,...,Z90710,NaN,NaN,NaN,NaN,NaN,871,0,23,1
130294,146868,NaN,10/10/2022 0:00,10/19/2022 0:00,3,1,1,R4182,A4159,0BH17EZ,...,F209,NaN,I071,NaN,Z9981,NaN,871,0,25,3


In [21]:
    # Create list of all diagnosis and procedure columns (from 01 to 25)
dgns_cols = [f'DGNSCD{str(i).zfill(2)}' for i in range(1, 26)]
prcdr_cols = [f'PRCDRCD{str(i).zfill(2)}' for i in range(1, 26)]
    
    # Check which columns actually exist in the dataframe
dgns_cols = [col for col in dgns_cols if col in df.columns]
prcdr_cols = [col for col in prcdr_cols if col in df.columns]
    
cols_to_drop = dgns_cols + prcdr_cols



In [22]:
for col in cols_to_drop:
    df = df.drop(col, axis=1)


In [23]:
df

,ID,STAY_DRG_CD,STAY_FROM_DT,STAY_THRU_DT,STUS_CD,TYPE_ADM,SRC_ADMS,AD_DGNS,stay_drg_cd,Readmitted_30,DGNS_COUNT,PRCDR_COUNT
0,17319,NaN,12/13/2017 0:00,12/20/2017 0:00,62,1,2,M25551,469,0,25,1
1,19722,NaN,10/19/2017 0:00,10/23/2017 0:00,1,1,1,R531,871,1,15,0
2,89699,NaN,8/6/2018 0:00,8/8/2018 0:00,1,1,1,R002,177,0,21,0
3,8086,NaN,12/20/2016 0:00,12/27/2016 0:00,62,5,1,K661,393,0,22,0
4,68049,NaN,1/6/2016 0:00,1/12/2016 0:00,6,1,1,J9601,177,0,13,0
...,...,...,...,...,...,...,...,...,...,...,...,...
130291,119880,NaN,12/18/2018 0:00,12/19/2018 0:00,1,3,2,R079,247,0,7,3
130292,103695,NaN,11/23/2019 0:00,11/27/2019 0:00,6,1,1,R531,291,1,24,7
130293,131933,NaN,12/10/2020 0:00,12/17/2020 0:00,6,1,1,R197,871,0,23,1
130294,146868,NaN,10/10/2022 0:00,10/19/2022 0:00,3,1,1,R4182,871,0,25,3


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130296 entries, 0 to 130295
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             130296 non-null  int64  
 1   STAY_DRG_CD    3798 non-null    float64
 2   STAY_FROM_DT   130296 non-null  object 
 3   STAY_THRU_DT   130296 non-null  object 
 4   STUS_CD        130296 non-null  int64  
 5   TYPE_ADM       130296 non-null  int64  
 6   SRC_ADMS       130296 non-null  int64  
 7   AD_DGNS        130296 non-null  object 
 8   stay_drg_cd    126498 non-null  object 
 9   Readmitted_30  130296 non-null  int64  
 10  DGNS_COUNT     130296 non-null  int64  
 11  PRCDR_COUNT    130296 non-null  int64  
dtypes: float64(1), int64(7), object(4)
memory usage: 11.9+ MB


In [25]:
df.isnull().sum()


ID                    0
STAY_DRG_CD      126498
STAY_FROM_DT          0
STAY_THRU_DT          0
STUS_CD               0
TYPE_ADM              0
SRC_ADMS              0
AD_DGNS               0
stay_drg_cd        3798
Readmitted_30         0
DGNS_COUNT            0
PRCDR_COUNT           0
dtype: int64

In [26]:
# Add new cell to analyze the STAY_DRG_CD null issue
print("STAY_DRG_CD null count:", df['STAY_DRG_CD'].isna().sum())
print("stay_drg_cd null count:", df['stay_drg_cd'].isna().sum())

# Check if the lowercase column (stay_drg_cd) has values where uppercase is null
if 'stay_drg_cd' in df.columns:
    # Check if we can fill values
    filled_count = df['STAY_DRG_CD'].isna() & df['stay_drg_cd'].notna()
    print(f"Number of rows where STAY_DRG_CD is null but stay_drg_cd has values: {filled_count.sum()}")
    
    # Fill the null values in STAY_DRG_CD with values from stay_drg_cd
    df['STAY_DRG_CD'] = df['STAY_DRG_CD'].fillna(df['stay_drg_cd'])
    print("After filling, STAY_DRG_CD null count:", df['STAY_DRG_CD'].isna().sum())
else:
    print("stay_drg_cd column not found")

STAY_DRG_CD null count: 126498
stay_drg_cd null count: 3798
Number of rows where STAY_DRG_CD is null but stay_drg_cd has values: 126498
After filling, STAY_DRG_CD null count: 0
